In [4]:
import pandas as pd
import os 

raw_df = pd.DataFrame(os.listdir('./Cleaned/'), columns=['image_name'])

def convert_format(df):
    split_names = df.image_name.apply(lambda x: x.split('_')).apply(lambda x: [x[0], ' '.join(x[1:-1]), x[-1]])
    df['split_name'] = split_names
    df = pd.DataFrame(df['split_name'].to_list(), columns=['age', 'name_id', 'image_id'])
    return df

df = convert_format(raw_df)

In [36]:
df

,age,name_id,image_id
0,14,0,0001.jpg
1,14,0,0002.jpg
2,14,10,0008.jpg
3,14,10,0009.jpg
4,14,11,0002.jpg
...,...,...,...
163394,62,1999,0003.jpg
163395,62,1999,0004.jpg
163396,62,1999,0005.jpg
163397,62,1999,0006.jpg


In [75]:
import random
def generate_samples(count):
    sampled_data = []
    for i in range(count):
        name_id_groups = df.groupby('name_id')
        name_ids = df['name_id'].unique().tolist()
        pos_id = random.sample(name_ids, 1)[0]
        table = name_id_groups.get_group(pos_id)
        common_ages = table.age.unique()
        for common_age in common_ages:
            anchor = df[(df['age'] == common_age) & (df['name_id'] == pos_id)].sample().values.tolist()[0]
            positive = df[(df['age'] != common_age) & (df['name_id'] == pos_id)].sample().values.tolist()[0]
            negative = df[(df['age'] == common_age) & (df['name_id'] != pos_id)].sample().values.tolist()[0]
            sampled_data.append(['_'.join(anchor), '_'.join(positive), '_'.join(negative)])
    sampled_df = pd.DataFrame(sampled_data, columns=['anchor', 'positive', 'negative'])
    return sampled_df

In [76]:
# testing
generate_samples(1)

,anchor,positive,negative
0,25_557_0009.jpg,30_557_0007.jpg,25_577_0002.jpg
1,26_557_0014.jpg,34_557_0009.jpg,26_227_0006.jpg
2,27_557_0004.jpg,34_557_0007.jpg,27_624_0016.jpg
3,28_557_0005.jpg,29_557_0002.jpg,28_361_0009.jpg
4,29_557_0009.jpg,33_557_0006.jpg,29_518_0001.jpg
5,30_557_0004.jpg,29_557_0014.jpg,30_418_0011.jpg
6,31_557_0005.jpg,30_557_0007.jpg,31_678_0005.jpg
7,32_557_0003.jpg,25_557_0003.jpg,32_842_0012.jpg
8,33_557_0007.jpg,26_557_0007.jpg,33_641_0005.jpg
9,34_557_0018.jpg,26_557_0005.jpg,34_1026_0015.jpg


##### Generate samples

In [77]:
generate_samples(10000)